In [2]:
%matplotlib qt

import os
import re
from scipy.io import loadmat
import mat73

import pandas as pd
import numpy as np
import mne

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
TIMELINE_F = '/mnt/ernie_main/000_Data/SEEG/SEEG_Periictal/data/Extracted_Per_Event_Interictal/all_time_data_01042023_212306.csv'
SEEG_FOLDER = '/mnt/ernie_main/000_Data/SEEG/SEEG_Entire_EMU_Downloads/data/'

In [5]:
def load_mat(f):
    try:
        return loadmat(f)
    except IOError:
        return mat73.loadmat(f)

seeg_timeline_df = pd.read_csv(TIMELINE_F, delimiter="\t")
seeg_timeline_df.head(5)

,Type,Pat ID,Event Number (Matches EMU Final Report),Date (MM:DD:YYYY),Onset String (HH:MM:SS),Offset String (HH:MM:SS),Seizure Type (FAS; FIAS; FBTC; Non-electrographic; Subclinical; Unknown),FileIndex,onset_datetime,offset_datetime,onset_datetime_buffer,offset_datetime_buffer,FileName
0,File,Epat02,File1,09:14:2017,13:05:11,17:05:11,NaN,0,2017-09-14 13:05:11,2017-09-14 17:05:11,2017-09-14 13:06:11,2017-09-14 17:04:11,Epat02_09142017_13051100.EDF
1,File,Epat02,File2,09:14:2017,17:05:22,21:05:22,NaN,1,2017-09-14 17:05:22,2017-09-14 21:05:22,2017-09-14 17:06:22,2017-09-14 21:04:22,Epat02_09142017_17052200.EDF
2,File,Epat02,File3,09:14:2017,21:05:34,01:05:34,NaN,2,2017-09-14 21:05:34,2017-09-15 01:05:34,2017-09-14 21:06:34,2017-09-15 01:04:34,Epat02_09142017_21053400.EDF
3,File,Epat02,File4,09:15:2017,01:05:45,05:05:45,NaN,3,2017-09-15 01:05:45,2017-09-15 05:05:45,2017-09-15 01:06:45,2017-09-15 05:04:45,Epat02_09152017_01054500.EDF
4,File,Epat02,File5,09:15:2017,05:05:56,09:05:56,NaN,4,2017-09-15 05:05:56,2017-09-15 09:05:56,2017-09-15 05:06:56,2017-09-15 09:04:56,Epat02_09152017_05055600.EDF


In [6]:
pat = 'Epat02'
pat_df = seeg_timeline_df[seeg_timeline_df['Pat ID'] == pat]
pat_edf = os.path.join(SEEG_FOLDER, pat, pat_df.FileName.values[0])
pat_seeg = mne.io.read_raw(pat_edf)
pat_seeg.info.keys()

Extracting EDF parameters from /mnt/ernie_main/000_Data/SEEG/SEEG_Entire_EMU_Downloads/data/Epat02/Epat02_09142017_13051100.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])

In [7]:
tmp = pat_seeg.crop(tmin=100, tmax=400)

# Preprocessing, toy example

## Bipolar Re-referencing

In [8]:
ch_names = pat_seeg.info['ch_names']
ch_names = [c for c in ch_names if re.search('[0-9]', c) != None] 
lead_names = set([re.sub('[0-9]+',"", c) for c in ch_names])

In [9]:
lead_names
drop = ['DC','EEG Mark', 'EKG']
lead_names = lead_names.difference(drop)

In [10]:
def make_bipole_list(lead_names, monopole_ch):
    bipole_channels = []
    for lead in lead_names:
        ch_list = [c for c in monopole_ch if re.sub('[0-9]+', "", c) == lead]
        if len(ch_list) % 2 == 0:
            bips =  [(ch_list[i], ch_list[i+1]) for i in range(0, len(ch_list), 2)]
        else:
            bips = [(ch_list[i], ch_list[i+1]) for i in range(0, len(ch_list)-1, 2)]
            bips.append((ch_list[-2], ch_list[-1]))
        bipole_channels.extend(bips)
    return bipole_channels

In [11]:
bip_list = make_bipole_list(lead_names, ch_names)
pat_seeg.load_data()
for b1, b2 in bip_list:
    pat_seeg = mne.set_bipolar_reference(pat_seeg, anode=b1, cathode = b2, drop_refs=False)
for ch in ch_names:
    pat_seeg.drop_channels(ch)

Reading 0 ... 150000  =      0.000 ...   300.000 secs...
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=150001
    Range : 50000 ... 200000 =    100.000 ...   400.000 secs
Ready.
Added the following bipolar channels:
LH1-LH2
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=150001
    Range : 50000 ... 200000 =    100.000 ...   400.000 secs
Ready.
Added the following bipolar channels:
LH3-LH4
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=150001
    Range : 50000 ... 200000 =    100.000 ...   400.000 secs
Ready.
Added the following bipolar channels:
LH5-LH6
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=150001
    Range : 50000 ... 200000 =    100.000 ...   400.000 secs
Ready.
Added the following bipolar channels:
LH7-LH8
EEG channel type selected for re-referencin

In [12]:
pat_seeg.info['ch_names']


['E',
 'Cz',
 'Pz',
 'Events/Markers',
 'LH1-LH2',
 'LH3-LH4',
 'LH5-LH6',
 'LH7-LH8',
 'LH8-LH9',
 'L56-L57',
 'L58-L59',
 'L60-L61',
 'L62-L63',
 'L63-L64',
 'RI1-RI2',
 'RI3-RI4',
 'RI5-RI6',
 'RI7-RI8',
 'RI9-RI10',
 'RH1-RH2',
 'RH3-RH4',
 'RH5-RH6',
 'RH7-RH8',
 'RH8-RH9',
 'RA1-RA2',
 'RA3-RA4',
 'RA5-RA6',
 'RA7-RA8',
 'RA8-RA9',
 'LAC1-LAC2',
 'LAC3-LAC4',
 'LAC5-LAC6',
 'LAC7-LAC8',
 'LAC9-LAC10',
 'RAC1-RAC2',
 'RAC3-RAC4',
 'RAC5-RAC6',
 'RAC7-RAC8',
 'RAC9-RAC10',
 'B49-B50',
 'B51-B52',
 'B53-B54',
 'B55-B56',
 'B57-B58',
 'B59-B60',
 'B61-B62',
 'B63-B64',
 'LOF1-LOF2',
 'LOF3-LOF4',
 'LOF5-LOF6',
 'LOF7-LOF8',
 'LOF9-LOF10',
 'ROF1-ROF2',
 'ROF3-ROF4',
 'ROF5-ROF6',
 'ROF7-ROF8',
 'ROF9-ROF10',
 'LI1-LI2',
 'LI3-LI4',
 'LI5-LI6',
 'LI7-LI8',
 'LI9-LI10',
 'LA1-LA4',
 'LA5-LA6',
 'LA7-LA8',
 'LA9-LA10',
 'LA2-LA3']

In [13]:
tmp = pat_seeg.notch_filter(freqs=[60,120,180,240])

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:    0.1s finished


In [19]:
tmp.filter(l_freq= 200, h_freq=1)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 1 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 200.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 225.00 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:    0.1s finished


<RawEDF | Epat02_09142017_13051100.EDF, 67 x 150001 (300.0 s), ~76.7 MB, data loaded>

# Calculating Connectivity Between Channels



In [ ]:
##TODO look into pypdc documentation 